<a href="https://colab.research.google.com/github/rainbowcity05/tibame0410/blob/main/pchome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import urllib.request as req
import bs4 as bs
import os
import pandas as pd

#建立資料夾
dn = "pchome/"
if not os.path.exists(dn):
    os.mkdir(dn)

#準備table
table = []

url = "https://24h.pchome.com.tw/search/?q=%E5%90%B9%E9%A2%A8%E6%A9%9F"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())
links = html.find_all("a", {"class":"c-prodInfoV2__link"})
for link in links:
    link_href = "https://24h.pchome.com.tw" + link["href"]
    p_resp = req.urlopen(link_href)
    p_html = bs.BeautifulSoup(p_resp.read())

    # 產品品牌
    p_brand = p_html.find("span", {"class":"o-prodMainName__colorSecondary"}).text

    # 產品名稱
    p_name = p_html.find("h1", {"class":"o-prodMainName__grayDarkest"}).text
    p_name_text = p_name.strip()

    #評分
    p_rating_all = p_html.find("div", {"class":"c-ratingIcon__flex"})
    if not p_rating_all == None:
        p_rating = p_rating_all.find("div", {"class":"c-ratingIcon__textNumber"})
        p_rating_count = p_rating_all.find("a")
        p_rating_text = p_rating.text.strip()
        p_rating_count_text = p_rating_count.text.strip().replace("則評價", "")
    else:
        p_rating_text = "-"
        p_rating_count_text = "-"
    #介紹
    p_intro_list = p_html.find_all("li", {"class":"c-blockCombine__item--prodSlogan"})
    p_intro_text_list = []
    for p_intro in p_intro_list:
        p_intro_text = p_intro.text.strip()
        p_intro_text_list.append(p_intro_text)
    p_intro_text_join = ""
    for p_intro_text in p_intro_text_list:
        p_intro_text_join = p_intro_text_join + p_intro_text + "\n"

    #價格
    p_discount_price = p_html.find("div", {"class":"o-prodPrice__price"}).text
    p_original_price = p_html.find("div", {"class":"o-prodPrice__originalPrice"})
    if not p_original_price == None:
      p_original_price = p_original_price.text
    else:
      p_original_price = "-"

    #抓圖片url
    p_img = p_html.find("div", {"class":"c-radiusPhotoImage__img"}).find("img")
    p_img_source = p_img["src"]
    if not "https:" in p_img_source:
        p_img_source = "https:" + p_img_source
    #圖片存檔
    fn = link_href.split("/")[-1]
    fn = dn + fn +".jpg"
    req.urlretrieve(p_img_source,fn)


    #滿額贈
    p_labels = p_html.find_all("div", {"class":"c-label__rectangle"})
    p_full_prize = "N"
    for p_label in p_labels:
        if "滿額贈" in p_label.text:
            p_full_prize = "Y"


    # print(p_brand)
    # print(p_name)
    # print(p_rating_text)
    # print("幾則評價:",p_rating_count_text)
    # print(p_intro_text_join)
    # print("折扣價:",p_discount_price)
    # print("原價:",p_original_price)
    # print(p_full_prize)
    # print(p_img_source)
    # print("*"*30)


    #建立raw data
    data = {
      "品牌":p_brand,
      "名稱":p_name,
      "評分":p_rating_text,
      "幾則":p_rating_count_text,
      "介紹":p_intro_text_join,
      "現價":p_discount_price,
      "原價":p_original_price,
      "滿額贈":p_full_prize,
      "圖片":p_img_source,
      "圖片儲存":fn
    }
    table.append(table)

df = pd.DataFrame(table)
df.to_csv("pchome.csv", encoding="utf-8", index=False)
df




Traceback (most recent call last):
